In [1]:
# Imports
import sys
sys.path.append("..")
from os import listdir
from os.path import isfile, join
import h5py
import os
import numpy as np
import torch
import torch.nn as nn 
import torch.optim as optim
import torch.nn.functional as F
import torch.backends.cudnn as cudnn
from torch.autograd import Variable
from torch.utils.data import Dataset
import MinkowskiEngine as ME
import MinkowskiEngine.MinkowskiFunctional as MF

/home/lauri/anaconda3/lib/python3.8/site-packages/MinkowskiEngine/__init__.py:36: UserWarning: The environment variable `OMP_NUM_THREADS` not set. MinkowskiEngine will automatically set `OMP_NUM_THREADS=16`. If you want to set `OMP_NUM_THREADS` manually, please export it on the command line before running a python script. e.g. `export OMP_NUM_THREADS=12; python your_program.py`. It is recommended to set it below 24.
  warnings.warn(


In [2]:
if torch.cuda.is_available():
    device = torch.device('cuda')
else:
    device = torch.device('cpu')
       
# Paths
DATA_PATH = os.path.abspath("../data/h5/")

# Model Variables 
BATCH_SIZE = 1
NUM_CLASSES = 2
NUM_EPOCHS = 20
LEARNING_RATE = 0.001
L2 = 0.005
MOMENTUM = 0.9
QUANTIZATION_SIZE=0.005

In [3]:
device="cpu"
print(device)

cpu


In [4]:
# Custom dataset
class SparseDataset(Dataset):
    def __init__(self, data_dir,  quantization_size=0.005):
        self.data_dir = data_dir
        self.quantization_size=quantization_size
        self.data_files = [f for f in listdir(data_dir) if isfile(join(data_dir, f))]

    def __len__(self):
        return len(self.data_files)

    def __getitem__(self, idx):
        data_path = os.path.join(self.data_dir, self.data_files[idx])
        with h5py.File(data_path, "r") as hf:
            coords = hf['coords'][:]
            features = hf['feats'][:]
            labels = np.int_(hf['labels'][:])
        discrete_coords, unique_feats, unique_labels = ME.utils.sparse_quantize(
            coordinates=coords,
            features=features,
            labels=labels,
            quantization_size=self.quantization_size)
        return discrete_coords, unique_feats, unique_labels


In [5]:
# Create dataset and dataloader
train = SparseDataset(DATA_PATH, quantization_size=QUANTIZATION_SIZE)
train_loader = torch.utils.data.DataLoader(train, batch_size = BATCH_SIZE, collate_fn=ME.utils.batch_sparse_collate, num_workers=1)

In [6]:
class ConvBlock(ME.MinkowskiNetwork):
    def __init__(self, in_channels, out_channels, D=3):
        super(ConvBlock, self).__init__(D)
        self.block = nn.Sequential(
            ME.MinkowskiBatchNorm(in_channels),
            ME.MinkowskiReLU(),
            ME.MinkowskiConvolution(in_channels, out_channels, kernel_size=3, stride=1, dimension=D),
            ME.MinkowskiBatchNorm(out_channels),
            ME.MinkowskiReLU(),
            ME.MinkowskiConvolution(out_channels, out_channels, kernel_size=3, stride=1, dimension=D)
        )
    def forward(self, x):
        return self.block(x)

class MiniSeg(ME.MinkowskiNetwork):
    def __init__(self, in_channels, n_classes, D=3):
        super(MiniSeg, self).__init__(D)
        self.pool = ME.MinkowskiMaxPooling(2, dimension=D)
        self.SM = ME.MinkowskiSoftmax()
        self.c1 = ConvBlock(in_channels, 32, D)
        self.c2 = ConvBlock(32, 64, D)
        self.c3 = ConvBlock(64, 128, D)
        
        self.t1 = ME.MinkowskiConvolutionTranspose(128, 64, kernel_size=3, stride=1, dimension=D)
        self.c4 = ConvBlock(128, 64, D)
        self.t2 = ME.MinkowskiConvolutionTranspose(64, 32,kernel_size=3, stride=1, dimension=D)
        self.c5 = ConvBlock(64, 32,D)
        
        self.out = ME.MinkowskiConvolution(32, n_classes, kernel_size=1, dimension=D)
    def forward(self, x):
        cat1 = self.c1(x)
        o = self.pool(cat1)
        cat2 = self.c2(o)
        o = self.pool(cat2)
        
        o = self.c3(o)
        
        o = self.t1(o)
        o = ME.cat(cat2, o)
        o = self.c4(o)
        o = self.t2(o)
        o = ME.cat(cat1, o)
        o = self.c5(o)
        
        o = self.out(o)
        return o


In [7]:
# Model
 
model = MiniSeg(2, 2, D=3).to(device)
loss_func = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=LEARNING_RATE, weight_decay=L2, momentum=MOMENTUM)


In [8]:
# Training 
for epoch in range(NUM_EPOCHS):
    total_loss = 0
    for i, data in enumerate(train_loader): 
        print("Processing tile ",i)
        coords, feats, labels = data
        optimizer.zero_grad()
        #print(coords)
        #print(feats)
        #print(labels)
        y_ = model(ME.SparseTensor(feats.float(), coords, device=device))
        loss = loss_func(y_.F.squeeze(), labels.long())
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
        del coords, feats, labels,y_
        torch.cuda.empty_cache()
    print(f"Epoch {epoch}: loss {total_loss}")


Processing tile  0
Processing tile  1
Processing tile  2
Processing tile  3
Processing tile  4
Processing tile  5
Processing tile  6
Processing tile  7
Processing tile  8
Processing tile  9
Processing tile  10
Processing tile  11
Processing tile  12
Processing tile  13
Processing tile  14
Processing tile  15
Processing tile  16
Processing tile  17
Processing tile  18
Processing tile  19
Processing tile  20
Processing tile  21
Processing tile  22
Processing tile  23
Processing tile  24
Processing tile  25
Processing tile  26
Epoch 0: loss 18.721812188625336
Processing tile  0
Processing tile  1
Processing tile  2
Processing tile  3
Processing tile  4
Processing tile  5
Processing tile  6
Processing tile  7
Processing tile  8
Processing tile  9
Processing tile  10
Processing tile  11
Processing tile  12
Processing tile  13
Processing tile  14
Processing tile  15
Processing tile  16
Processing tile  17
Processing tile  18
Processing tile  19
Processing tile  20
Processing tile  21
Process

KeyboardInterrupt: 